# 01 Visualize Signals（やさしいチュートリアル版）

このノートブックは、**EMA（移動平均）** と **ATR（ボラティリティ）** を使って、
「今はどんな地合いか？」「クロスのシグナルが出たか？」を順番に理解するためのミニ教科書です。  

> ゴール：読みながら動かし、パラメータをいじって「なぜこの判定になるか」を自分で説明できるようになる。

## Glossary（用語集）

| Term | Meaning | In sensor analogy |
|---|---|---|
| OHLC | 1本の足の始値・高値・安値・終値。 | センサーの1周期での最初/最大/最小/最後の観測値。 |
| EMA | 直近データを重めに平均したトレンド線。 | 低周波だけ通すローパスフィルタ。 |
| EMA cross | 短期EMAと長期EMAの交差イベント。 | 短い時定数と長い時定数の出力が入れ替わる瞬間。 |
| True Range (TR) | ギャップ込みの1本あたり変動幅。 | 1サンプルでの実効振幅。 |
| ATR | TRの平均で見た変動強度。 | 振幅（ノイズレベル）の移動平均。 |
| Regime / Active | 相場状態ラベル。ここではATRがしきい値超えならActive。 | センサー状態が「静穏/活性」のどちらか。 |


## 0) Parameter knobs（最初に調整するつまみ）

最初にこのセルだけ編集すれば、銘柄・期間・各種パラメータを一括で変更できます。


In [ ]:
# --- Parameter knobs: まずはここをいじる ---
SYMBOL = "1306.T"      # 例: "7203.T", "AAPL"
PERIOD = "2y"          # 取得期間（例: "6mo", "1y", "2y", "5y"）
INTERVAL = "1d"        # 足種（このノートでは日足を想定）

EMA_FAST = 12           # 短期EMAのspan
EMA_SLOW = 26           # 長期EMAのspan
ATR_PERIOD = 14         # ATR期間
REGIME_WIN = 60         # Active判定しきい値（ATR移動中央値）の窓
LAST_N_ROWS = 12        # 最後に表示する行数（学習用テーブル）


### Parameter knobs の意味

- `SYMBOL`: 対象銘柄（何を観測するか）。
- `PERIOD`: 取得する履歴の長さ（どれだけ過去を見るか）。
- `EMA_FAST`: 短期EMAの感度。小さいほど反応が速い。
- `EMA_SLOW`: 長期EMAの感度。大きいほど滑らか。
- `ATR_PERIOD`: 変動幅を平均する期間。
- `REGIME_WIN`: Active判定しきい値の計算窓。

センサー比喩では、`EMA_FAST/EMA_SLOW` はフィルタの時定数、`ATR_PERIOD/REGIME_WIN` は振幅監視の平均窓です。


## 1) Data loading（データ取得）

In [ ]:
from quantlab.data import fetch_ohlc
from quantlab.indicators import ema, atr
from quantlab.plot import plot_price_ema, plot_atr_regime, plot_cross_points
import pandas as pd

# データ取得
# fetch_ohlc は通常 OHLCV を含むDataFrameを返します。
df = fetch_ohlc(SYMBOL, period=PERIOD, interval=INTERVAL).copy()

# 指標計算（列名は後続セルでも使うため先に作成）
df["EMA_FAST"] = ema(df["Close"], EMA_FAST)
df["EMA_SLOW"] = ema(df["Close"], EMA_SLOW)
df["ATR"] = atr(df, ATR_PERIOD)

# EMA差分（クロス判定の基本）
df["ema_diff"] = df["EMA_FAST"] - df["EMA_SLOW"]

df.tail(3)

## 2) Data sanity check（データ健全性チェック）

分析に入る前に、次を確認します。
- 冒頭/末尾の行（head/tail）
- カラム一覧
- 日付範囲
- NaN 件数（指標計算の最初にNaNが出るのは自然）

In [ ]:
# 先頭と末尾を確認
print("=== df.head() ===")
display(df.head())
print("=== df.tail() ===")
display(df.tail())

# カラム一覧
print("\n=== columns ===")
print(df.columns.tolist())

# 日付範囲（DatetimeIndex想定。違う場合にも対応）
if isinstance(df.index, pd.DatetimeIndex):
    start_date = df.index.min()
    end_date = df.index.max()
else:
    # indexが日時でない場合は、Date列などを探すためのフォールバック
    start_date = df.index.min()
    end_date = df.index.max()
print("\n=== date range ===")
print(f"start: {start_date}")
print(f"end  : {end_date}")

# NaN 件数
print("\n=== NaN counts ===")
print(df.isna().sum())

## 3) 指標の意味を式と数値で確認（OHLC / EMA / TR / ATR / EMA cross）

### OHLC
**What it measures（何を測る？）**: 1本のローソク足の価格レンジと着地です。  
**Why we use it here（このノートで使う理由）**: すべての指標計算の入力で、まず観測データ自体を確認するためです。  
センサー比喩: OHLCは1サイクル中の原波形の要約です。

### EMA
**What it measures（何を測る？）**: 価格のなめらかな方向（トレンド）です。  
**Why we use it here（このノートで使う理由）**: 短期と長期を比較して方向転換の候補を作るためです。  
センサー比喩: EMAはローパスフィルタで、短期EMAは高感度・長期EMAは安定です。

$$
\begin{aligned}
EMA_t &= \alpha\,C_t + (1-\alpha)\,EMA_{t-1} \\
\alpha &= \frac{2}{N+1}
\end{aligned}
$$

### EMA cross
**What it measures（何を測る？）**: 短期EMAと長期EMAの優位が入れ替わる瞬間です。  
**Why we use it here（このノートで使う理由）**: 売買シグナル候補の発火点を定義するためです。  
センサー比喩: 2つの異なる時定数フィルタの出力交差は状態切替のトリガーです。

$$
\begin{aligned}
d_t &= EMA^{(fast)}_t - EMA^{(slow)}_t \\
\text{cross\_up}_t &\Leftrightarrow (d_{t-1}\le 0)\land(d_t>0) \\
\text{cross\_down}_t &\Leftrightarrow (d_{t-1}\ge 0)\land(d_t<0)
\end{aligned}
$$

### True Range と ATR
**What it measures（何を測る？）**: TRは1本の実効変動幅、ATRはその平均強度です。  
**Why we use it here（このノートで使う理由）**: クロスが出ても「動いている局面か」を見分けるためです。  
センサー比喩: ATRは信号振幅（ノイズレベル）メータです。

$$
\begin{aligned}
TR_t &= \max\{H_t-L_t,\ |H_t-C_{t-1}|,\ |L_t-C_{t-1}|\} \\
ATR_t &= \mathrm{SMA}_{N}(TR)_t
\end{aligned}
$$


In [ ]:
# --- Numeric example: OHLC / TR / ATR / EMA / EMA cross（latest values） ---
# OHLC latest
print("=== Numeric example: OHLC (latest) ===")
print(df[["Open", "High", "Low", "Close"]].tail(1))

# TRとATRの計算確認
tr_example = df[["High", "Low", "Close"]].copy()
tr_example["C_prev"] = tr_example["Close"].shift(1)
tr_example["H-L"] = tr_example["High"] - tr_example["Low"]
tr_example["|H-C_prev|"] = (tr_example["High"] - tr_example["C_prev"]).abs()
tr_example["|L-C_prev|"] = (tr_example["Low"] - tr_example["C_prev"]).abs()
tr_example["TR_manual"] = tr_example[["H-L", "|H-C_prev|", "|L-C_prev|"]].max(axis=1)
tr_example["ATR_manual"] = tr_example["TR_manual"].rolling(ATR_PERIOD).mean()

print(f"\n=== Numeric example: TR / ATR (latest, ATR_PERIOD={ATR_PERIOD}) ===")
display(tr_example[["H-L", "|H-C_prev|", "|L-C_prev|", "TR_manual", "ATR_manual"]].tail(1))

atr_gap = (tr_example["ATR_manual"] - df["ATR"]).dropna()
print("ATR manual - implementation (latest):", atr_gap.iloc[-1] if len(atr_gap) else "NA")

# EMA更新式の確認
alpha_fast = 2 / (EMA_FAST + 1)
ema_example = df[["Close", "EMA_FAST"]].copy()
ema_example["EMA_prev"] = ema_example["EMA_FAST"].shift(1)
ema_example["EMA_formula"] = alpha_fast * ema_example["Close"] + (1 - alpha_fast) * ema_example["EMA_prev"]
ema_example["formula_gap"] = ema_example["EMA_FAST"] - ema_example["EMA_formula"]

print(f"\n=== Numeric example: EMA (latest, EMA_FAST={EMA_FAST}) ===")
display(ema_example[["Close", "EMA_prev", "EMA_formula", "EMA_FAST", "formula_gap"]].tail(1))

# クロス判定の数値確認
cross_example = df[["EMA_FAST", "EMA_SLOW", "ema_diff"]].copy()
cross_example["ema_diff_prev"] = cross_example["ema_diff"].shift(1)
cross_example["crossed_up"] = (cross_example["ema_diff_prev"] <= 0) & (cross_example["ema_diff"] > 0)
cross_example["crossed_down"] = (cross_example["ema_diff_prev"] >= 0) & (cross_example["ema_diff"] < 0)

print("\n=== Numeric example: EMA cross (latest) ===")
display(cross_example.tail(1))


## 4) Regime / Active を式と数値で確認

**What it measures（何を測る？）**: 相場を「静穏」か「活性」かの状態ラベルで表します。  
**Why we use it here（このノートで使う理由）**: クロスシグナルを、動きが十分ある局面だけに絞るためです。  
センサー比喩: regimeは装置状態（idle/active）のステートです。

$$
\begin{aligned}
\theta_t &= \mathrm{Median}_{W}(ATR)_t \\
\text{active}_t &= \mathbb{1}[ATR_t > \theta_t]
\end{aligned}
$$


In [ ]:
# ATRのしきい値（移動中央値）
df["atr_threshold"] = df["ATR"].rolling(REGIME_WIN).median()

# Active regime の真偽
df["active"] = df["ATR"] > df["atr_threshold"]

latest_atr = df["ATR"].iloc[-1]
latest_threshold = df["atr_threshold"].iloc[-1]
latest_active = bool(df["active"].iloc[-1])

print("=== Active regime (latest) ===")
print(f"ATR{ATR_PERIOD} latest : {latest_atr:.4f}")
print(f"median{REGIME_WIN} threshold: {latest_threshold:.4f}")
print(f"active: {latest_active}")

active_ratio = df["active"].mean() * 100
print(f"\n% of days active in dataset: {active_ratio:.2f}%")

## 5) Crossover signal（最終シグナル）

**What it measures（何を測る？）**: クロスの向きと active 条件を通過した最終イベントです。  
**Why we use it here（このノートで使う理由）**: 「どの日を候補と見なすか」を機械的に再現するためです。  
センサー比喩: 交差トリガーに状態ゲートを掛けたイベント検出器です。

$$
\begin{aligned}
signal_t &= +1 && \text{if cross\_up}_t \land \text{active}_t \\
signal_t &= -1 && \text{if cross\_down}_t \land \text{active}_t \\
signal_t &= 0 && \text{otherwise}
\end{aligned}
$$


In [ ]:
# Boolean logic を明示
crossed_up = (df["ema_diff"].shift(1) <= 0) & (df["ema_diff"] > 0)
crossed_down = (df["ema_diff"].shift(1) >= 0) & (df["ema_diff"] < 0)

# active フィルタ込みのシグナル
#   +1: BUY候補（上抜け）
#   -1: SELL候補（下抜け）
#    0: シグナルなし
df["signal"] = 0
df.loc[crossed_up & df["active"], "signal"] = 1
df.loc[crossed_down & df["active"], "signal"] = -1

print("=== logic preview (last 5) ===")
display(
    pd.DataFrame({
        "ema_diff_prev<=0": (df["ema_diff"].shift(1) <= 0),
        "ema_diff_today>0": (df["ema_diff"] > 0),
        "crossed_up": crossed_up,
        "ema_diff_prev>=0": (df["ema_diff"].shift(1) >= 0),
        "ema_diff_today<0": (df["ema_diff"] < 0),
        "crossed_down": crossed_down,
    }).tail(5)
)

# 学習用の最終テーブル
cols = ["Close", "EMA_FAST", "EMA_SLOW", "ema_diff", "ATR", "active", "signal"]
print(f"\n=== last {LAST_N_ROWS} rows ===")
display(df[cols].tail(LAST_N_ROWS))

## 6) Plot 1: Price + EMA

**この図が示すもの**: 終値と短期/長期EMAの位置関係。  
**読み取り方**: 短期EMAが長期EMAの上で推移するか、交差直後かを確認します。  

考える質問:
- 直近で短期EMAと長期EMAの距離は拡大中か、縮小中か？
- 交差が出た直後に価格が追随しているか？

センサー比喩: 原信号（価格）に2つのローパス出力を重ねて位相差を見る図です。


In [ ]:
# 既存プロット関数をそのまま利用
plot_price_ema(df)

## 7) Plot 2: ATR regime

**この図が示すもの**: ATRとしきい値（移動中央値）、および active 区間。  
**読み取り方**: ATRがしきい値を上回る期間だけを「活性状態」として扱います。  

考える質問:
- 直近は active が続いているか、それとも途切れやすいか？
- クロスが出た日が active 区間に入っているか？

センサー比喩: 振幅メータが閾値を超えたときだけ動作する状態監視です。


In [ ]:
plot_atr_regime(df)

## 8) Plot 3: Cross points

**この図が示すもの**: 価格上での上抜け/下抜けの発生点。  
**読み取り方**: どのクロスが最終シグナルとして採用されたかを時系列で追います。  

考える質問:
- シグナル直後の数本で価格は同方向へ進んだか？
- 除外されたクロスは inactive 区間に偏っていないか？

センサー比喩: 状態ゲート付きイベント検出の発火ログです。


In [ ]:
plot_cross_points(df)